In [ ]:
!pip3 install geopy

In [5]:
from geopy.geocoders import Nominatim
import pandas as pd
import time

df = pd.read_csv("filtered_BexarMay24_hotel_tax_receipts.csv")
geolocator = Nominatim(user_agent="tx_hotels_map")

# Function to get coordinates
def get_coordinates(row):
    full_address = f"{row['Location_Address']}, {row['Location_City']}, TX {row['Location_Zip']}"
    try:
        location = geolocator.geocode(full_address)
        time.sleep(1)  # Be kind to the API
        if location:
            return pd.Series([location.latitude, location.longitude])
    except:
        return pd.Series([None, None])

# Apply to DataFrame
df[["Latitude", "Longitude"]] = df.apply(get_coordinates, axis=1)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Taxpayer_Number            614 non-null    int64  
 1   Taxpayer_Name              614 non-null    object 
 2   Taxpayer_Address           614 non-null    object 
 3   Taxpayer_City              614 non-null    object 
 4   Taxpayer_State             614 non-null    object 
 5   Taxpayer_Zip               614 non-null    int64  
 6   Taxpayer_County            614 non-null    int64  
 7   Taxpayer_Phone             614 non-null    object 
 8   Location_Number            614 non-null    int64  
 9   Location_Name              614 non-null    object 
 10  Location_Address           614 non-null    object 
 11  Location_City              614 non-null    object 
 12  Location_State             614 non-null    object 
 13  Location_Zip               614 non-null    int64  

In [ ]:
!pip3 install folium

In [6]:
import folium

# Create the base map centered on San Antonio
hotel_map = folium.Map(location=[29.5187, -98.6047], zoom_start=12)

# Normalize receipts to set radius scale
min_receipt = df["Total_Room_Receipts"].min()
max_receipt = df["Total_Room_Receipts"].max()
range_receipt = max_receipt - min_receipt

# Add scaled circle markers
for _, row in df.dropna(subset=["Latitude", "Longitude"]).iterrows():
    # Scale radius between 4 and 20
    scaled_radius = 4 + 16 * ((row["Total_Room_Receipts"] - min_receipt) / range_receipt)

    # Color by receipt amount (simple logic: low = light, high = dark blue)
    if row["Total_Room_Receipts"] == 0:
        color = "gray"
    elif row["Total_Room_Receipts"] < 100000:
        color = "lightblue"
    elif row["Total_Room_Receipts"] < 300000:
        color = "blue"
    else:
        color = "darkblue"

    popup_text = f"""
    <b>{row['Location_Name']}</b><br>
    Receipts: ${row['Total_Room_Receipts']:,.2f}<br>
    Units: {row['Unit_Capacity']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=scaled_radius,
        color=color,
        fill=True,
        fill_color=color,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(hotel_map)

# Save the map
hotel_map.save("Bexar_May24_receipts_scaled_map.html")
